In [2]:
import requests
from bs4 import BeautifulSoup
import os
import polars as pl
from zipfile import ZipFile
from datetime import datetime
from tqdm import tqdm
import time

import utils.source

from utils.request_market_data import (
    request_prices_to_landing,
    write_prices_to_raw,
    read_prices_from_lake,
    trasnform_trusted_prices
)

from utils.request_cvm_ciasabertas import (
    download_files,
    read_files_from_landing,
    write_files_to_lake,
    read_files_from_lake,
    trasnform_trusted_accounts,
    trasnform_trusted_tickers
)


In [2]:
read_prices_from_lake('raw', 'vale3').schema

OrderedDict([('date', Date),
             ('open', Float64),
             ('high', Float64),
             ('low', Float64),
             ('close', Float64),
             ('adjusted_close', Float64),
             ('volume', Int64),
             ('dividend_amount', Float64),
             ('split_coefficient', Float64)])

In [3]:
df = read_files_from_lake(zone='refined', table='active_companies')

In [4]:
prices = trasnform_trusted_prices(df['STOCK'])

100%|██████████| 450/450 [00:00<00:00, 986.96it/s]


In [5]:
write_files_to_lake(
    df = prices,
    zone = 'trusted',
    table = 'prices'
)

In [21]:
x

date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,STOCK
date,f64,f64,f64,f64,f64,i64,f64,f64,str
2019-05-20,23.01,24.48,23.0,23.8,23.8,6600,0.0,1.0,"""SEDU3"""
2019-05-17,23.3,23.3,23.3,23.3,23.3,0,0.0,1.0,"""SEDU3"""
2019-05-16,23.08,23.9,23.08,23.9,23.9,5000,0.0,1.0,"""SEDU3"""
2019-05-15,25.06,25.06,23.6,23.6,23.6,10000,0.0,1.0,"""SEDU3"""
2019-05-14,24.48,24.55,24.21,24.54,24.54,11200,0.0,1.0,"""SEDU3"""
2019-05-13,24.55,24.55,23.15,24.55,24.55,7900,0.0,1.0,"""SEDU3"""
2019-05-10,22.97,23.5,22.7,23.48,23.48,8000,0.0,1.0,"""SEDU3"""
2019-05-09,23.24,23.4,23.02,23.2,23.2,20700,0.0,1.0,"""SEDU3"""
2019-05-08,23.97,23.97,23.03,23.1,23.1,24700,0.0,1.0,"""SEDU3"""
